# Fed Minutes AI Analysis Demo

This notebook demonstrates Phase 3 AI-powered analysis capabilities:
- **RAG (Retrieval-Augmented Generation)** for intelligent Q&A
- **Historical research** with LLM-powered insights
- **Time period analysis** and trend detection
- **Topic evolution** tracking over time

## Setup and Configuration

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()  # This loads your OPENAI_API_KEY from .env file

import sys
import os
sys.path.append('..')

# Core imports
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from src.utils.config import load_config
from src.phase3_ai_analysis import create_rag_pipeline, create_llm_client
from src.phase2_knowledge_base import create_search_interface

import pandas as pd
from datetime import datetime
import json

# Verify API key is loaded
api_key_status = bool(os.getenv('OPENAI_API_KEY'))
print(f'✅ API key loaded from .env: {api_key_status}')
if not api_key_status:
    print('⚠️  No OPENAI_API_KEY found in .env file - will use mock responses')

In [ ]:
# Load configuration
config = load_config()
print("Configuration loaded successfully")
print(f"LLM Provider: {config.get('llm', {}).get('provider', 'openai')}")
print(f"Model: {config.get('llm', {}).get('model', 'gpt-4')}")

# Show available LLM options
print("\n📋 Available LLM Providers:")
print("- OpenAI: Requires OPENAI_API_KEY in .env")
print("- Anthropic: Requires ANTHROPIC_API_KEY in .env")  
print("- Ollama: Local models, no API key required")
print("- Mock: Test mode without any LLM")

# Check for Ollama if configured
if config.get('llm', {}).get('provider') == 'ollama':
    print(f"\n🦙 Ollama Configuration:")
    print(f"   Model: {config.get('llm', {}).get('model')}")
    print(f"   Base URL: {config.get('llm', {}).get('base_url')}")
    print("   Note: Make sure Ollama is running with your model installed")

## Initialize AI Analysis System

**Note**: This demo supports multiple LLM providers:

### 🌐 **API-Based Providers** (Require API Keys):
- **OpenAI**: Set `OPENAI_API_KEY` in your `.env` file
- **Anthropic**: Set `ANTHROPIC_API_KEY` in your `.env` file

### 🦙 **Local Provider** (No API Key Required):
- **Ollama**: Install Ollama and download a model:

  **On macOS:**
  ```bash
  # Option 1: Using Homebrew
  brew install ollama
  
  # Option 2: Download from https://ollama.com/download/mac
  
  # After installation, pull a model:
  ollama pull mistral:7b
  ```
  
  **On Linux:**
  ```bash
  curl -fsSL https://ollama.ai/install.sh | sh
  ollama pull mistral:7b
  ```
  
  **Then edit config/config.yaml:**
  ```yaml
  llm:
    provider: "ollama"
    model: "mistral:7b"
  ```

### 🧪 **Test Mode**:
- **Mock**: Uses mock responses for testing without any LLM

In [ ]:
# Initialize the RAG pipeline
print("Initializing AI Analysis System...")
rag = create_rag_pipeline(config)

# Check what LLM provider we're using
print(f"\nLLM Client: {rag.llm.__class__.__name__}")
print(f"Available: {rag.llm.is_available()}")

if rag.llm.__class__.__name__ == "MockLLMClient":
    print("\n⚠️  Using mock LLM responses for demo purposes")
    print("   Set OPENAI_API_KEY or ANTHROPIC_API_KEY for real analysis")
else:
    print(f"\n✅ Real LLM analysis available with {rag.llm.__class__.__name__}")

print("\nAI Analysis System ready!")

## 🦙 Optional: Configure for Ollama (Local LLM)

If you want to use a local model instead of OpenAI/Anthropic, you can configure Ollama:

In [ ]:
# OPTIONAL: Test Ollama configuration
# Uncomment and run this cell to use Ollama instead of OpenAI/Anthropic

# # Override config to use Ollama
# config['llm'] = {
#     'provider': 'ollama',
#     'model': 'mistral:7b',  # or 'llama3:8b', 'phi3:mini'
#     'temperature': 0.1,
#     'max_tokens': 1000,
#     'base_url': 'http://localhost:11434'
# }

# # Test Ollama availability
# from src.phase3_ai_analysis import OllamaClient
# ollama_test = OllamaClient(config['llm']['model'])
# if ollama_test.is_available():
#     print(f"✅ Ollama is available with model: {config['llm']['model']}")
#     print("   You can now run all the analysis cells with local LLM!")
# else:
#     print("❌ Ollama not available. To fix:")
#     print("   1. Install Ollama: https://ollama.ai/download")
#     print(f"   2. Run: ollama pull {config['llm']['model']}")
#     print("   3. Make sure Ollama is running")

# # Reinitialize RAG with Ollama
# rag = create_rag_pipeline(config)
# print(f"\nRAG pipeline reinitialized with {rag.llm.__class__.__name__}")

## 1. Intelligent Q&A with RAG

Ask questions about Fed Minutes content and get AI-powered answers with citations.

In [ ]:
# Example 1: General policy question
question = "What were the Federal Reserve's main concerns about inflation in 1971?"
print(f"Question: {question}\n")

response = rag.answer_question(
    question=question,
    date_range=("1971-01-01", "1971-12-31"),
    max_context_chunks=5
)

print(f"Answer: {response.answer}\n")
print(f"Confidence: {response.confidence:.2f}")
print(f"Citations: {len(response.citations)} meetings referenced")
print(f"Tokens used: {response.tokens_used}\n")

# Show citations
if response.citations:
    print("Meeting References:")
    for citation in response.citations[:3]:  # Show first 3
        print(f"  - {citation['meeting']} ({citation['date']})")
    if len(response.citations) > 3:
        print(f"  ... and {len(response.citations) - 3} more")

In [ ]:
# Example 2: Nixon Shock specific question
question = "How did the Federal Reserve respond to the Nixon Shock in August 1971?"
print(f"Question: {question}\n")

response = rag.answer_question(
    question=question,
    date_range=("1971-08-01", "1971-12-31"),
    max_context_chunks=6
)

print(f"Answer: {response.answer}\n")
print(f"Confidence: {response.confidence:.2f}")
print(f"Context chunks analyzed: {len(response.context.chunks)}")

In [ ]:
# Example 3: Bretton Woods question
question = "What did the Fed discuss about international monetary system changes in 1972-1973?"
print(f"Question: {question}\n")

response = rag.answer_question(
    question=question,
    date_range=("1972-01-01", "1973-12-31"),
    max_context_chunks=5
)

print(f"Answer: {response.answer}\n")
print(f"Confidence: {response.confidence:.2f}")

## 2. Period Summary Analysis

Generate comprehensive summaries of Fed discussions during specific time periods.

In [ ]:
# Summary of Nixon Shock period
print("Generating summary for Nixon Shock period (Aug-Dec 1971)...\n")

summary = rag.summarize_period(
    start_date="1971-08-01",
    end_date="1971-12-31",
    topics=["monetary policy", "nixon shock", "exchange rates", "international"],
    max_chunks=8
)

print(f"Period Summary (Aug-Dec 1971):")
print(f"{'='*50}")
print(f"{summary.answer}\n")

print(f"Analysis based on {len(summary.context.chunks)} meeting excerpts")
print(f"Confidence: {summary.confidence:.2f}")
print(f"Meetings referenced: {len(summary.citations)}")

In [ ]:
# Summary of Bretton Woods collapse period
print("Generating summary for Bretton Woods collapse period (1972-1973)...\n")

summary = rag.summarize_period(
    start_date="1972-01-01",
    end_date="1973-12-31",
    topics=["bretton woods", "international monetary", "exchange rates", "gold"],
    max_chunks=10
)

print(f"Period Summary (1972-1973):")
print(f"{'='*50}")
print(f"{summary.answer}\n")

print(f"Analysis based on {len(summary.context.chunks)} meeting excerpts")
print(f"Confidence: {summary.confidence:.2f}")

## 3. Topic Evolution Analysis

Track how Fed discussions of specific topics evolved over time.

In [ ]:
# Analyze evolution of inflation discussions
print("Analyzing evolution of inflation discussions (1969-1973)...\n")

evolution = rag.analyze_topic_evolution(
    topic="inflation price stability",
    start_year=1969,
    end_year=1973,
    chunks_per_year=3
)

print(f"Topic Evolution: Inflation Discussions (1969-1973)")
print(f"{'='*60}")
print(f"{evolution.answer}\n")

print(f"Analysis covers {evolution.context.search_params['years']}")
print(f"Total meeting excerpts analyzed: {len(evolution.context.chunks)}")
print(f"Confidence: {evolution.confidence:.2f}")

In [ ]:
# Analyze evolution of international monetary discussions
print("Analyzing international monetary system discussions (1970-1973)...\n")

evolution = rag.analyze_topic_evolution(
    topic="international monetary system exchange rates",
    start_year=1970,
    end_year=1973,
    chunks_per_year=4
)

print(f"Topic Evolution: International Monetary System (1970-1973)")
print(f"{'='*65}")
print(f"{evolution.answer}\n")

print(f"Total excerpts analyzed: {len(evolution.context.chunks)}")
print(f"Confidence: {evolution.confidence:.2f}")

## 4. Comparative Period Analysis

Compare Fed discussions between different time periods.

In [ ]:
# Compare pre and post Nixon Shock periods
print("Comparing Fed discussions: Pre vs Post Nixon Shock...\n")

comparison = rag.compare_periods(
    period1=("1970-01-01", "1971-07-31"),  # Pre-Nixon Shock
    period2=("1971-08-15", "1972-06-30"),  # Post-Nixon Shock
    aspects=["monetary policy", "inflation", "international", "exchange rates"]
)

print(f"Period Comparison: Pre vs Post Nixon Shock")
print(f"{'='*50}")
print(f"Pre-Nixon Shock: 1970-01-01 to 1971-07-31")
print(f"Post-Nixon Shock: 1971-08-15 to 1972-06-30")
print(f"\n{comparison.answer}\n")

print(f"Analysis based on {len(comparison.context.chunks)} meeting excerpts")
print(f"Confidence: {comparison.confidence:.2f}")
print(f"Meetings referenced: {len(comparison.citations)}")

## Summary and Next Steps

This notebook demonstrates the core capabilities of Phase 3 AI analysis:

### What We've Built:
✅ **Intelligent Q&A System**: Ask natural language questions about Fed Minutes content  
✅ **Period Summarization**: Generate comprehensive summaries of Fed discussions during specific timeframes  
✅ **Topic Evolution**: Track how Fed thinking evolved on key topics over time  
✅ **Comparative Analysis**: Compare Fed discussions between different periods  
✅ **Citation System**: All answers include references to source meetings  
✅ **Confidence Scoring**: Quality assessment for each analysis  
✅ **Local LLM Support**: Use Ollama for free, unlimited analysis without API keys

### Key Features:
- **RAG Pipeline**: Combines semantic search with LLM analysis
- **Multi-Provider Support**: Works with OpenAI, Anthropic, Ollama, or mock responses
- **Temporal Analysis**: Date-aware search and analysis
- **Research-Ready**: Designed for academic and policy research
- **Privacy Option**: Keep all data local with Ollama

### LLM Provider Comparison:
| Provider | Cost | Privacy | Speed | Quality | Setup |
|----------|------|---------|-------|---------|-------|
| OpenAI | $ per use | Cloud | Fast | Excellent | API key |
| Anthropic | $ per use | Cloud | Fast | Excellent | API key |
| Ollama | Free | Local | Medium | Good | Install app |
| Mock | Free | Local | Instant | Mock | None |

### Try These Research Questions:
- "How did the Fed view international cooperation during the Bretton Woods crisis?"
- "What were the main disagreements in Fed meetings during 1971?"
- "How did Fed concerns about unemployment change from 1969 to 1973?"
- "What role did William McChesney Martin play in monetary policy decisions?"

### Phase 4 Vision:
- **Automated Research Reports**: Generate comprehensive analysis documents
- **Pattern Recognition**: Discover hidden relationships and trends
- **Decision Tree Analysis**: Map Fed decision-making processes
- **Interactive Dashboard**: Web interface for researchers

In [ ]:
# Interactive Q&A - modify this cell to ask your own questions

# Your question here:
your_question = "What did the Fed discuss about wage and price controls?"
date_range = ("1971-01-01", "1972-12-31")  # Optional date filter

print(f"Your Question: {your_question}")
if date_range:
    print(f"Date Range: {date_range[0]} to {date_range[1]}")
print()

response = rag.answer_question(
    question=your_question,
    date_range=date_range,
    max_context_chunks=6
)

print(f"Answer: {response.answer}\n")
print(f"Confidence: {response.confidence:.2f}")
print(f"Context: {len(response.context.chunks)} chunks analyzed")

# Show some context for verification
if response.context.chunks:
    print("\nSample context (first excerpt):")
    first_chunk = response.context.chunks[0]
    print(f"Meeting: {first_chunk['filename']} ({first_chunk['date'][:10]})")
    print(f"Preview: {first_chunk['chunk_text'][:200]}...")

## Summary and Next Steps

This notebook demonstrates the core capabilities of Phase 3 AI analysis:

### What We've Built:
✅ **Intelligent Q&A System**: Ask natural language questions about Fed Minutes content  
✅ **Period Summarization**: Generate comprehensive summaries of Fed discussions during specific timeframes  
✅ **Topic Evolution**: Track how Fed thinking evolved on key topics over time  
✅ **Comparative Analysis**: Compare Fed discussions between different periods  
✅ **Citation System**: All answers include references to source meetings  
✅ **Confidence Scoring**: Quality assessment for each analysis  

### Key Features:
- **RAG Pipeline**: Combines semantic search with LLM analysis
- **Multi-Provider Support**: Works with OpenAI, Anthropic, or mock responses
- **Temporal Analysis**: Date-aware search and analysis
- **Research-Ready**: Designed for academic and policy research

### Try These Research Questions:
- "How did the Fed view international cooperation during the Bretton Woods crisis?"
- "What were the main disagreements in Fed meetings during 1971?"
- "How did Fed concerns about unemployment change from 1969 to 1973?"
- "What role did William McChesney Martin play in monetary policy decisions?"

### Phase 4 Vision:
- **Automated Research Reports**: Generate comprehensive analysis documents
- **Pattern Recognition**: Discover hidden relationships and trends
- **Decision Tree Analysis**: Map Fed decision-making processes
- **Interactive Dashboard**: Web interface for researchers